In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from time import sleep
import csv

In [14]:
# Open Chrome and access to Linkedin
driver = webdriver.Chrome(ChromeDriverManager().install())
url = "https://www.linkedin.com/"
driver.get(url)
print('-Finish initializing a driver')

# Prevent timeout
driver.implicitly_wait(20) # gives an implicit wait for 20 seconds

# Import username and password
credential = open('id.txt')
line = credential.readlines()
username = line[0]
password = line[1]
print('-Finish importing the login credential')

# Key in username
email_field = driver.find_element_by_id("session_key")
email_field.send_keys(username)
sleep(2)

# Key in password
password_field = driver.find_element_by_id("session_password")
password_field.send_keys(password)
sleep(3)

# Click the Sign in button
login_field = driver.find_element_by_xpath(r'''//*[@id="main-content"]/section[1]/div[2]/form/button''')
login_field.click()
print('-Finish logging in')
sleep(3)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\USER\.wdm\drivers\chromedriver\win32\94.0.4606.41\chromedriver.exe] found in cache


-Finish initializing a driver
-Finish importing the login credential
-Finish logging in


In [15]:
# Input the search query to search bar
search_field = driver.find_element_by_xpath(r'''//*[@id="global-nav-typeahead"]/input''')

search_query = input("What do you want to scrape? ")
search_field.send_keys(search_query)
sleep(2)

# Press Enter to search
search_field.send_keys(Keys.RETURN)
print("-Finish search")

# Read one page source and get URL of profile
def GetURL():
    page_source = BeautifulSoup(driver.page_source, features="html.parser")
    driver.implicitly_wait(10)

    # Find URL in page source 
    all_profile_URL = []
    profiles = page_source.find_all('a', class_='app-aware-link')
    for profile in profiles:
        profile_URL = profile.get('href')
        if profile_URL not in all_profile_URL:
            all_profile_URL.append(profile_URL)
    return all_profile_URL

# Loop and find in (number_of_page)

input_page = int(input('How many pages you want to scrape: '))
URLs_all_page = []
for page in range(input_page):
    URLs_one_page = GetURL()
    sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    driver.implicitly_wait(5)
    next_button = driver.find_element_by_class_name("artdeco-pagination__button--next")
    next_button.click()
    driver.implicitly_wait(5)
    URLs_all_page = URLs_all_page + URLs_one_page
    print('-Finish get URL one page')
    sleep(2)

print('-Finish getting all URLs')

What do you want to scrape? data analyst people
-Finish search
How many pages you want to scrape: 1
-Finish get URL one page
-Finish getting all URLs


In [ ]:
# Scrape data of Linkedin profile
# Write to csv file
with open('output.csv', mode='w', encoding='utf-8', newline = '') as file_output:
    headers = ['name', 'school', 'title', 'location', 'URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n', fieldnames=headers)
    writer.writeheader()

    for linkedin_URL in URLs_all_page:
        try:
            driver.get(linkedin_URL)
            sleep(2)
            page_source = BeautifulSoup(driver.page_source, features="html.parser")
            info_div = page_source.find('div', class_ = "display-flex justify-space-between pt2")

            name = info_div.find('h1').get_text().strip()
            print(name)

            school = info_div.find('h2').get_text().strip()
            print(school)

            loc_title = info_div.find_all(class_='text-body-medium break-words')
            for a in loc_title:
                title = a.get_text().strip()
            print(title)

            loc_location = info_div.find_all(class_='text-body-small inline t-black--light break-words')
            for b in loc_location:
                location = b.get_text().strip()
            print(location)
            print('\n')
            
            with open('output.csv', mode='a', encoding='utf-8', newline = '') as file_output:
                writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n', fieldnames=headers)
                writer.writerow({headers[0]:name, headers[1]:school, headers[2]:title, headers[3]:location, headers[4]:linkedin_URL})

        except (AttributeError):
            print('This URL is not a profile\n')
        except (WebDriverException):
            print('Webdriver is closed manually')

            

Diep Nguyen (Chloe)
RMIT University Vietnam
People Data Analyst
Vietnam


Brian O'Reilly
Vietnamese-German University
My passion is to help business people learn and develop their skills through my work in higher education and also honorary roles in the Vietnam Business Forum, AusCham, AmCham and EuroCham.
Ho Chi Minh City, Vietnam


